# List of active United States Military Aircraft

![Data Project Preview](https://jerichocruz.com/images/pbi/header.png)

This is a project to visualize a list of **active** military aircraft that are used by the United States military. This will not include aircraft that is no longer in-service.

### Contents

1. Introduction
   * 1.1 Background
   * 1.2. User Stories
2. Data Source
   * 2.1. Data Limitations
   * 2.2. Notes
   * 2.3. Does the data ROCCC?
3. Data Cleansing & Transformation (Python)
   * 3.1. Reading data in pandas
   * 3.2. Make changes to the data
   * 3.3. Filtering Data
   * 3.4. Write the dataframe to excel
4. Data Model
5. Active US Military Aircraft Dashboard
6. Analysis Results

## 1. Introduction

### 1.1. Background

For this analysis I play the part of a Data Analyst who works with the Aviation Accountability Office, a fictitious, independent, non-partisan agency that works for Congress. AAO provides Congress, the heads of executive agencies, and the public with timely, fact-based, non-partisan information that can be used to improve visibility in Military Aviation accountability. 

### 1.2. User Stories

The business request for this project was an executive military aircraft report showing active aircraft in use by the United States for the AAO. Based on the request that was made from the office, the following user stories were defined to fulfill delivery and ensure that acceptance criteria was maintained throughout the project.

| #	As a (role) | I want (request / demand) | So that I (user value) | Acceptance Criteria
| :- | :- | :- | :- 
| 1. Defense Capabilities and Management Executive | To get a dashboard overview of active US military aircraft | Can follow better what aircraft is in use | A Power BI dashboard which updates data upon refresh
| 2. Department Representative | A detailed overview of Aircraft per Branch | Can follow what aircraft is in use by each branch | A Power BI dashboard which allows me to filter data for each branch
| 3. Strategic Planner | A detailed overview of Aircraft per Manufacturer | Can follow up which Manufacturer and aircraft is most in use | A Power BI dashboard which allows me to filter data for each Manufacturer
| 4. Sustainment Analyst | A dashboard overview of Aircraft introduction dates | Follow aircraft in service over date introduced | A Power BI dashboard with graphs to filter by dates ascending

## 2. Data Source

- The data provided is a public data set located on [Wikipedia](https://en.wikipedia.org/wiki/List_of_active_United_States_military_aircraft)
- Generated and updated from aviation plans, databases and acquisition reports between March 2009 to August 2020
- 33 references are cited
- Supplemental: Data from the Sustainment Risk Report by GAO (Government Accountability Office)

### 2.1. Data Limitations

- Some citations are over 5+ years old
- Scheduled and in-progress replacements are not tracked, but are mentioned in sources
- Data set is limited to what is recorded on Wikipedia

### 2.2. Notes

- Aircraft is listed with original manufacturer(s); This means that a merger (e.g., Grumman Corp and Northrop Corp, Lockheed and Martin Marietta) will not change the name of the manufacturer


### 2.3. Does the data ROCCC?

- Reliable: Various Type/Models are not included (e.g., KC-130J Aerial Refueler used by U.S. Marines not present in the dataset)
- Original: Data was obtained via third party, Wikipedia
- Comprehensive: Aircraft used by multiple branches and made by different manufacturers are in the dataset
- Current: At the time of this report, the last update to the dataset was 26 June 2022
- Cited: 33 references are cited

Observation: Quality of the data in the data set is not accurate and should not be used to advise stakeholders of business solutions. Data should be used to provide a high level summary and may provide insights to perform deeper analysis.

## 3. Data Cleansing & Transformation (Python)

To create the necessary data model for doing analysis and fulfilling the business needs defined in the user stories, we will be using pandas, requests, re, klib, and beautifulsoup to webscrape the wikipedia page.

In [1]:
import pandas as pd
import requests
import numpy as np
import re
from bs4 import BeautifulSoup
import klib # !pip install -U klib

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_active_United_States_military_aircraft').text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table', class_="wikitable sortable")

df = pd.read_html(str(table))
df = pd.concat(df)
print(df.shape)

(162, 15)


### 3.1. Reading data in pandas

In [3]:
# Read headers
df.columns

Index(['Aircraft', 'Manufacturer', 'Origin', 'Propulsion', 'Role', 'Control',
       'Introduced', 'In service[1][2][3]', 'Total', 'Notes', 'Unnamed: 10',
       'Type', 'In service[15]', 'In service[22]', 'In service[23][24]'],
      dtype='object')

### 3.2. Make changes to the data

In [4]:
# Clean columns using klib library
df = klib.clean_column_names(df) # cleans and standardizes column names
df = klib.convert_datatypes(df) # converts existing to more efficient dtypes

df.columns
df.dtypes

aircraft                 string
manufacturer             string
origin                   string
propulsion             category
role                     string
control                category
introduced               object
in_service[1][2][3]      string
total                    object
notes                    string
unnamed_10              float32
type                     string
in_service[15]           string
in_service[22]          float32
in_service[23][24]       string
dtype: object

In [5]:
# Assign values from "type" column if "aircraft" is blank
df.aircraft = df.aircraft.replace('', pd.NA).fillna(df.type)
df.tail()

,aircraft,manufacturer,origin,propulsion,role,control,introduced,in_service[1][2][3],total,notes,unnamed_10,type,in_service[15],in_service[22],in_service[23][24]
25,MH-60 Seahawk,Sikorsky,USA,Helicopter,Anti-submarine warfare helicopterMulti-mission,Manned,1984,<NA>,NaN,<NA>,NaN,MH-60 Seahawk,508,NaN,<NA>
26,TH-57B/C Sea Ranger,Bell,USA,Helicopter,Trainer,Manned,1984,<NA>,NaN,To be replaced by the AgustaWestland TH-57A Th...,NaN,TH-57B/C Sea Ranger,114,NaN,<NA>
27,ScanEagle,Boeing,USA,Propeller,<NA>,Unmanned,2005,<NA>,NaN,<NA>,NaN,ScanEagle,<NA>,NaN,<NA>
28,RQ-21A Blackjack,Boeing Insitu,USA,Propeller,<NA>,Unmanned,2014,<NA>,NaN,<NA>,NaN,RQ-21A Blackjack,<NA>,NaN,<NA>
29,MQ-8B Fire Scout,Northrop Grumman,USA,Helicopter,Patrol,Unmanned,2009,<NA>,NaN,Helicopter. 96 planned.[33],NaN,MQ-8B Fire Scout,27[27],NaN,<NA>


In [6]:
# Review dtypes before working with re to clean the dataframe
df.dtypes

aircraft                 string
manufacturer             string
origin                   string
propulsion             category
role                     string
control                category
introduced               object
in_service[1][2][3]      string
total                    object
notes                    string
unnamed_10              float32
type                     string
in_service[15]           string
in_service[22]          float32
in_service[23][24]       string
dtype: object

In [7]:
# We see that one column is a "Float32", so we need to convert 'in_service[22]' column into a string
df['in_service[22]'] = df['in_service[22]'].astype('string')

df.dtypes

aircraft                 string
manufacturer             string
origin                   string
propulsion             category
role                     string
control                category
introduced               object
in_service[1][2][3]      string
total                    object
notes                    string
unnamed_10              float32
type                     string
in_service[15]           string
in_service[22]           string
in_service[23][24]       string
dtype: object

In [8]:
# Replace all NaN values with blank
df = df.replace(np.nan, '', regex=True)

In [9]:
# Combine in_service[x] columns into one
cols = ['in_service[1][2][3]', 'in_service[15]', 'in_service[22]', 'in_service[23][24]']
df['in_service'] = df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

# Remove square brackets that were intended for footnotes in the wiki; This is done by using a delimiter
df['in_service'] = df['in_service'].str.split('[').str[0]
df['role'] = df['role'].str.split('[').str[0]

# Drop columns
df = df.drop(columns=cols)
df.drop('type', axis=1, inplace=True) # This was merged with aircraft column
df.drop('unnamed_10', axis=1, inplace=True) # A blank column that is irrelevant

df.head()

,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service
0,A-10C Thunderbolt II,Fairchild Republic,USA,Jet,CAS / Attack,Manned,1977,,,281
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20
3,B-1B Lancer,Rockwell International,USA,Jet,Bomber,Manned,1986,,Employs variable-sweep wing design. To be repl...,45
4,B-2A Spirit,Northrop Grumman,USA,Jet,Bomber,Manned,1997,,Stealth capable aircraft. To be replaced by th...,19


In [10]:
# Create a new column called 'branch'
df['branch'] = ''

# Assign branch based on index location; 'branch' column is [10]
# !! Compare with wiki page in case something is added/removed (since this uses index loc) !!
df.iloc[0:66,[10]] = 'Air Force'
df.iloc[66:94,[10]] = 'Army'
df.iloc[94:104,[10]] = 'Coast Guard'
df.iloc[104:132,[10]] = 'Marines'
df.iloc[132:162,[10]] = 'Navy'

df


,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service,branch
0,A-10C Thunderbolt II,Fairchild Republic,USA,Jet,CAS / Attack,Manned,1977,,,281,Air Force
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6,Air Force
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20,Air Force
3,B-1B Lancer,Rockwell International,USA,Jet,Bomber,Manned,1986,,Employs variable-sweep wing design. To be repl...,45,Air Force
4,B-2A Spirit,Northrop Grumman,USA,Jet,Bomber,Manned,1997,,Stealth capable aircraft. To be replaced by th...,19,Air Force
...,...,...,...,...,...,...,...,...,...,...,...
25,MH-60 Seahawk,Sikorsky,USA,Helicopter,Anti-submarine warfare helicopterMulti-mission,Manned,1984,,,508,Navy
26,TH-57B/C Sea Ranger,Bell,USA,Helicopter,Trainer,Manned,1984,,To be replaced by the AgustaWestland TH-57A Th...,114,Navy
27,ScanEagle,Boeing,USA,Propeller,,Unmanned,2005,,,,Navy
28,RQ-21A Blackjack,Boeing Insitu,USA,Propeller,,Unmanned,2014,,,,Navy


In [11]:
# Filter out any letters in the 'introduced' column (ex. 1980s)
df.introduced = df.introduced.astype('string')
df.introduced = df.introduced.str.extract('([0-9]+[0-9]+[0-9]+[0-9])')

In [12]:
# Ensure that 'in_service' column is a string dtype
df.in_service = df.in_service.astype('string')

df.dtypes

aircraft          string
manufacturer      string
origin            string
propulsion      category
role              object
control         category
introduced        object
total             object
notes             string
in_service        string
branch            object
dtype: object

### 3.3. Filtering data

This is an example of how we can filter data in our dataframe in Python. However, I will be using Power Query in Power BI to perform more data cleansing and filtering.

In [13]:
# Example: Find Aircraft made by Lockheed
lockheed = df.loc[df['manufacturer'].str.contains("Lockheed")]
lockheed

,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service,branch
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6,Air Force
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20,Air Force
6,C-5M Super Galaxy,Lockheed,USA,Jet,Strategic airlifter,Manned,1970,,,52,Air Force
13,C-130H Hercules,Lockheed Martin,USA,Propeller,Tactical airlifter,Manned,1956,,,192,Air Force
14,C-130J Super Hercules,Lockheed Martin,USA,Propeller,Tactical airlifter,Manned,1996,,,132,Air Force
24,EC-130H Compass Call,Lockheed Martin,USA,Propeller,Radar jamming / PSYOP,Manned,1982,,,12,Air Force
25,EC-130J Commando Solo III,Lockheed Martin,USA,Propeller,Radar jamming / PSYOP,Manned,1978,,,7,Air Force
30,F-22A Raptor,Lockheed Martin,USA,Jet,Air superiority,Manned,2005,,Stealth capable aircraft. To be replaced by NGAD,186,Air Force
31,F-35A Lightning II,Lockheed Martin,USA,Jet,Multirole,Manned,2016,,Stealth capable aircraft. F-16 replacement,283,Air Force
32,HC-130J Combat King II,Lockheed Martin,USA,Propeller,Search and rescue,Manned,1959,,,32,Air Force


### 3.4. Write the dataframe to Excel

After preprocessing data using Python, we can convert the dataframe to a format that can be loaded into a visualization tool (e.g., Microsoft PowerBI).


In [14]:
df.to_excel('aircraft.xlsx', sheet_name='US Military Aircraft', index=False)

# Display first 50 rows
df.head(50)

,aircraft,manufacturer,origin,propulsion,role,control,introduced,total,notes,in_service,branch
0,A-10C Thunderbolt II,Fairchild Republic,USA,Jet,CAS / Attack,Manned,1977,,,281,Air Force
1,AC-130J Ghostrider,Lockheed,USA,Propeller,CAS / Attack,Manned,2017,,Replacement for the AC-130U.,6,Air Force
2,AC-130W Stinger II,Lockheed,USA,Propeller,CAS / Attack,Manned,1966,,Currently being replaced by the AC-130J.,20,Air Force
3,B-1B Lancer,Rockwell International,USA,Jet,Bomber,Manned,1986,,Employs variable-sweep wing design. To be repl...,45,Air Force
4,B-2A Spirit,Northrop Grumman,USA,Jet,Bomber,Manned,1997,,Stealth capable aircraft. To be replaced by th...,19,Air Force
5,B-52H Stratofortress,Boeing,USA,Jet,Bomber,Manned,1955,,2 retired B52H being reactivated to replace da...,74,Air Force
6,C-5M Super Galaxy,Lockheed,USA,Jet,Strategic airlifter,Manned,1970,,,52,Air Force
7,C-12C/D/H/J Huron,Beechcraft,USA,Propeller,Transport,Manned,1972,,32 aircraft are used for reconnaissance,55,Air Force
8,C-17A Globemaster III,McDonnell Douglas/Boeing,USA,Jet,Strategic airlifter,Manned,1995,,Were produced by McDonnell-Douglas prior to th...,222,Air Force
9,C-21A Learjet 35,Learjet,USA,Jet,VIP transport,Manned,1985,,,18,Air Force


## 4. Data Model

Below is a screenshot of the data model after cleansed and prepared tables were read into Power BI.

We can see that the FACT table (fAircraft) is connected to six DIMENSION tables with the correct relationship established (1 to *, 1 to 1) between dimension and fact tables.

![Power BI Data Model](https://jerichocruz.com/images/pbi/datamodel.png)


## 5. Active US Military Aircraft Dashboard

Click the image below to try out the dashboard!

[![Power BI Dashboard](https://jerichocruz.com/images/pbi/dashboard-prev.png)](https://app.powerbi.com/view?r=eyJrIjoiYjZhMGZmZDItMjVkMS00MTk4LTk2NDUtNmVkNGQ0YTI5YTY4IiwidCI6IjQzYjM3MWRiLWU5MzktNGZlOC1hYmI3LTlmZjI0YzgzMGNhYiJ9)

## 6. Analysis Results

What are some highlights and trends?

- Most used aircraft by the US Military is the UH-60 Black Hawk with 1443 in service by the Army
- Boeing is the Top Manufacturer/Provider of Aircraft (by type) to the Army and Navy
  - Marines: McDonnell Douglas
  - Army: Beechcraft
  - Coast Guard: Airbus
- While aircraft roles vary
  - Army and Navy are more likely use their aircraft for Cargo/Transport missions
  - Coast Guard: Search & Rescue
  - Air Force: Training
  - Marines: Utility
- 13.34k Aircraft currently in service with 971 Unmanned


How can these insights help AAO?

- AAO can tie this data with costs of sustainability and procurement
- AAO can use these trends to gather an idea of how each branch utilizes their aircraft
- Identify which manufacturer has market share over a department or mission type